In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install transformers==4.41.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 67.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [5]:
!pip install -U langchain-community
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Instal

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import T5Tokenizer, T5ForSequenceClassification, AutoTokenizer, AutoModel
import os
from tqdm import tqdm

In [7]:
# model for convert text to vectors
model_name = "sentence-transformers/sentence-t5-large"
embedding = HuggingFaceEmbeddings(model_name=model_name)


def process_and_return_vectorstore(filePath, savePath="", save=False):
  with open(filePath, "r", encoding="utf-8") as f:
    knowledge_points = [line.strip() for line in f.readlines() if line.strip()]

  docs = [Document(page_content=point, metadata={}) for point in knowledge_points]
  vectorstore = Chroma.from_documents(documents=docs[:10], embedding=embedding, persist_directory=savePath, collection_name="knowledge_collection")
  if save:
    vectorstore.persist()
  return vectorstore

def search_vectorstore_return_vector(query, vectorstore_hf, top_k=1):
  query_embedding = embedding.embed_query(query)
  similar_docs = vectorstore_hf.similarity_search_by_vector(query_embedding, k=top_k)
  context_embeddings = [doc.embedding for doc in similar_docs]
  context = torch.tensor(context_embeddings).unsqueeze(0)
  return context

def search_vectorstore_return_text(query, vectorstore_hf, top_k=1):
  similar_docs = vectorstore_hf.similarity_search(query, k=top_k)
  context_text = " ".join([doc.page_content for doc in similar_docs])
  return context_text



<ipython-input-7-c6618aa127fd>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [8]:
def get_evaluator_data(file):
  with_label = False
  # with_label = True
  content = []
  label = []
  with open(file, "r", encoding="utf-8") as f:
    if with_label:
      for line in f.readlines()[:]:
        c, l = line.split("\t")
        content.append(c)
        label.append((int(l.strip()) - 0.5) * 2)
      return content, label
    else:
      for line in f.readlines():
        content.append(line.strip())
      return content, None

def inference(tokenizer, model, file, device=torch.device("cpu"), n_docs=10):
  model.eval()
  content, label = get_evaluator_data(file)

  preds = []
  scores = []

  for n, c in tqdm(enumerate(content[:])):
    if c.strip().endswith('[SEP]'):
      preds.append(-1)
      scores.append(-1.0)
      continue
    if n % 10 > n_docs-1:
      continue
    test = tokenizer(c, return_tensors="pt",padding="max_length",max_length=512)
    with torch.no_grad():
      outputs = model(test["input_ids"].to(device),attention_mask=test["attention_mask"].to(device))
    pred_flat = 1 if outputs["logits"].cpu() > 0 else -1
    scores.append(float(outputs["logits"].cpu()))
    preds.append(pred_flat)
  return scores

def data_preprocess(file, n_docs):
  # with_label = True
  with_label = False
  queries = []
  passages = []
  tmp_psgs = []
  with open(file, "r", encoding="utf-8") as f:
    if with_label:
      for i, line in enumerate(f.readlines()[:]):
        c, l = line.strip().split("\t")
        q, p = c.split(' [SEP] ')
        if queries == []:
          queries.append(q)
          tmp_psgs = [p]
        else:
          if q != queries[-1]:
            passages.append(' [sep] '.join(tmp_psgs[:n_docs]))
            queries.append(q)
            tmp_psgs = [p]
          else:
            tmp_psgs.append(p)
        passages.append(' [sep] '.join(tmp_psgs[:n_docs]))
    else:
      for i, line in enumerate(f.readlines()):
        c = line.strip()
        if c.endswith('[SEP]'):
          c += ' '
        q, p = c.split(' [SEP] ')
        if queries == []:
          queries.append(q)
          tmp_psgs = [p]
        else:
          if q != queries[-1]:
            passages.append(' [sep] '.join(tmp_psgs[:n_docs]))
            queries.append(q)
            tmp_psgs = [p]
          else:
            tmp_psgs.append(p)
      passages.append(' [sep] '.join(tmp_psgs[:n_docs]))
  return queries, passages

def CRAG_Inference(data_path, evaluator_path, device, LLMmodel, LLMtokenizer, input_file, output_file, ndocs, threshold1, threshold2):
  correct = process_and_return_vectorstore(os.path.join(data_path, "correct"))
  incorrect = process_and_return_vectorstore(os.path.join(data_path, "incorrect"))
  ambiguous = process_and_return_vectorstore(os.path.join(data_path, "ambiguous"))

  tokenizer = T5Tokenizer.from_pretrained(evaluator_path)
  model = T5ForSequenceClassification.from_pretrained(evaluator_path, num_labels=1)
  device = torch.device(device) if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)
  queries, passages = data_preprocess(input_file, ndocs)
  print("processing evaluator")
  scores = inference(
    tokenizer=tokenizer,
    model=model,
    file=input_file,
    device=device,
    n_docs=ndocs
  )
  print("corrective processing")
  preds = []
  for score, query in tqdm(zip(scores, queries)):
    content_text = ""
    if score >= threshold1:
      content_text = search_vectorstore_return_text(query, correct, top_k=1)
    elif score >= threshold2:
      content_text = search_vectorstore_return_text(query, ambiguous, top_k=1)
    else:
      content_text = search_vectorstore_return_text(query, incorrect, top_k=1)

    prompt = f"Knowledge for refference：{content_text}\nQuestion：{query}\nAnswer："
    response, history = LLMmodel.chat(LLMtokenizer, prompt, history=[])

    #inputs = LLMtokenizer(prompt, return_tensors="pt")
    #outputs = LLMmodel.generate(**inputs, max_length=150)
    #answer = LLMtokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)
    preds.append(response)

  with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(preds))


In [ ]:
import transformers
print(transformers.__version__)


4.41.2


In [9]:
# chatglm for testing
chatglm_model = AutoModel.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True).half().cuda()
chatglm_model = chatglm_model.eval()
chatglm_tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

configuration_chatglm.py:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_chatglm.py:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

quantization.py:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [11]:
# prepare medical data  data_path/correct  data_path/incorrect  data_path/ambiguous
data_path = "./data"
evaluator_path = "/content/drive/MyDrive/Colab Notebooks/finetuned_t5_evaluator"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_file = "./data/input.txt"
output_file = "./data/output_medical.txt"
ndocs = 10
thread1 = 0.592
thread2 = 0.995

CRAG_Inference(data_path, evaluator_path, device, chatglm_model, chatglm_tokenizer, input_file, output_file, ndocs, thread1, thread2)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


processing evaluator


50it [00:00, 512751.10it/s]


corrective processing


1it [00:05,  5.74s/it]

...

vertigo is a spinning sensation, often accompanied by dizziness, that can occur when you change the position of your head. It is the most common cause of vertigo, and it can be caused by a variety of factors, including inner ear problems or conflicts between the inner and outer ear. In some cases, vertigo can be a sign of a more serious underlying condition, such as a brain tumor or an aneurysm. However, in most cases, vertigo is not serious and can be treated with medication or other forms of therapy. In Benign paroxysmal positional vertigo (BPPV), the vertigo occurs due to the inner ear's inability to keep the balance of the body and the head. BPPV is generally not serious but can disrupt daily activities and increase the risk of falls, so it is important to seek medical attention if you experience symptoms of BPPV.


4it [00:23,  6.38s/it]

To prevent positional vertigo, you can try the following methods:

1. Avoid abrupt head movements: Try not to turn your head abruptly or quickly, as this can trigger vertigo.

2. Relax and lie down: Take a break and lie down if you feel dizzy. This can help you avoid sudden changes in body position.

3. Avoid sudden changes in body: Try not to change your body position too quickly or suddenly, as this can also cause vertigo.

4. Regular physical therapy exercises: Engage in regular physical therapy exercises like the Epley maneuver, which can help reduce the likelihood of vertigo episodes.

5. Reduce salt intake: Avoid consuming too much salt as it can lead to dehydration, which can trigger vertigo.

6. Stay hydrated: Drink plenty of water to stay hydrated and prevent dehydration, which can cause vertigo.

7. Get enough rest: Aim for adequate sleep and rest to help your body recover from stress and prevent vertigo.

Remember, if your vertigo is severe or persists for an extended period

5it [00:27,  5.44s/it]

For benign paroxysmal positional vertigo (BPPV), there are no specific medications to take. The treatment options primarily involve lifestyle modifications and specific exercises or maneuvers to help relieve the symptoms. These may include avoiding triggers such as rapid head movements, maintaining a balanced diet, and following a regular exercise routine. In some cases, medication may be prescribed to help manage underlying conditions that may contribute to BPPV, such as low blood pressure or inner ear disorders. It is best to consult with a healthcare professional for personalized advice and treatment options.


In [ ]:
data_path = "./data"
evaluator_path = "/content/drive/MyDrive/Colab Notebooks/finetuned_t5_evaluator"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
input_file = "./data/test.txt"
output_file = "./data/output.txt"
ndocs = 10
thread1 = 0.592
thread2 = 0.995

CRAG_Inference(data_path, evaluator_path, device, chatglm_model, chatglm_tokenizer, input_file, output_file, ndocs, thread1, thread2)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


processing evaluator


1000it [01:06, 15.02it/s]


corrective processing


1it [00:04,  4.12s/it]

...

Maurice Le Boucher was a lawyer.


99it [02:06,  1.35it/s]

T. P. Poonatchi is not mentioned in the provided reference as an occupation.


100it [02:07,  1.28s/it]

I'm sorry, but I couldn't find any information about a person named René Bégin. Could you please provide more context or clarify the name you are asking about?
